In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn .metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [15]:
df = pd.read_csv('../data/curated/predict_data.csv')

In [17]:
df = df.iloc[:, [1,4,5,6,7,8,9,10,11,12,13,14]]
dummies = pd.get_dummies(df.furnished, prefix = ['f'])
df = pd.concat([df, dummies],axis = 1)
del df['furnished']

# calculate population change each year
df['pop23-22'] = df['2023 no.'] - df['2022 no.']
df['pop24-23'] = df['2024 no.'] - df['2023 no.']
df['pop25-24'] = df['2025 no.'] - df['2024 no.']
df = df.drop(columns = ['2022 no.', '2023 no.', '2024 no.', '2025 no.'])

y = df['price']
x = df.drop(columns = ['postcode', 'price', 'pop24-23', 'pop25-24'], axis = 1)
#x_list = list(x.columns)
#x = np.array(x)


scaler = MinMaxScaler()
scaler.fit(x)
x_scaler = scaler.transform(x)

x_train, x_test, y_train, y_test = train_test_split(x_scaler, y, test_size = 0.28, random_state = 42)

In [19]:
from sklearn.neural_network import MLPRegressor

print ("fitting model right now")
fit1 = MLPRegressor(hidden_layer_sizes=(100,50), activation='relu',solver='lbfgs',alpha=0.01,max_iter=1000).fit(x_train,y_train)

pred1_train = fit1.predict(x_train)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error

mse_1 = mean_squared_error(pred1_train,y_train)
print ("Train ERROR = ", mse_1)

pred1_test = fit1.predict(x_test)
mse_2 = mean_squared_error(pred1_test,y_test)
print("Test ERROR = ", mse_2)

mae1 = mean_absolute_error(pred1_train,y_train)
print("Train MAE = ", mae1)

mae2 = mean_absolute_error(pred1_test,y_test)
print("Test MAE = ", mae2)

print('Train MedianAE = ',median_absolute_error(pred1_train,y_train))
print('Test MedianAE = ',median_absolute_error(pred1_test,y_test))

fitting model right now
Train ERROR =  6415.83793783979
Test ERROR =  17901.293186537037
Train MAE =  57.00066843801619
Test MAE =  68.03999086654785
Train MedianAE =  42.45883399730042
Test MedianAE =  43.212484219179544


/Users/Shron/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [20]:
pre_price = fit1.predict(x_scaler)
df['price_2023'] = pre_price
df

,postcode,price,bed,bath,parking,cbd_dist,school_counts,['f']_0,['f']_1,pop23-22,pop24-23,pop25-24,price_2023
0,3141,400.0,1,1,1.0,5.9614,6.0,1,0,-7.0,4.0,8.0,434.459686
1,3182,360.0,1,1,1.0,4.1882,5.0,1,0,151.0,219.0,170.0,383.762866
2,3141,320.0,1,1,0.0,6.9114,6.0,1,0,-7.0,4.0,8.0,388.895639
3,3163,360.0,1,1,1.0,14.3864,5.0,1,0,266.0,282.0,280.0,327.921215
4,3182,320.0,1,1,1.0,5.8505,5.0,1,0,151.0,219.0,170.0,377.294283
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3427,3754,350.0,2,1,1.0,37.3377,10.0,1,0,5.0,6.0,17.0,431.572485
3428,3186,510.0,2,1,1.0,8.7118,8.0,1,0,225.0,255.0,260.0,500.945105
3429,3181,550.0,2,1,0.0,7.0455,3.0,1,0,598.0,524.0,367.0,531.730320
3430,3122,600.0,2,2,1.0,9.1834,8.0,1,0,221.0,369.0,590.0,522.031497


In [21]:
# new y is the price of 2023
y2 = df['price_2023']
x2 = df.iloc[:, [1, 10]]

x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, test_size = 0.28, random_state = 42)
fit2 = MLPRegressor(hidden_layer_sizes=(100,50), activation='relu',solver='lbfgs',alpha=0.01,max_iter=1000).fit(x2_train,y2_train)
y2_pred = fit2.predict(x2)
df['price_2024'] = y2_pred
df

/Users/Shron/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


,postcode,price,bed,bath,parking,cbd_dist,school_counts,['f']_0,['f']_1,pop23-22,pop24-23,pop25-24,price_2023,price_2024
0,3141,400.0,1,1,1.0,5.9614,6.0,1,0,-7.0,4.0,8.0,434.459686,402.722694
1,3182,360.0,1,1,1.0,4.1882,5.0,1,0,151.0,219.0,170.0,383.762866,374.474120
2,3141,320.0,1,1,0.0,6.9114,6.0,1,0,-7.0,4.0,8.0,388.895639,349.129574
3,3163,360.0,1,1,1.0,14.3864,5.0,1,0,266.0,282.0,280.0,327.921215,377.099980
4,3182,320.0,1,1,1.0,5.8505,5.0,1,0,151.0,219.0,170.0,377.294283,358.552649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3427,3754,350.0,2,1,1.0,37.3377,10.0,1,0,5.0,6.0,17.0,431.572485,369.260675
3428,3186,510.0,2,1,1.0,8.7118,8.0,1,0,225.0,255.0,260.0,500.945105,445.206938
3429,3181,550.0,2,1,0.0,7.0455,3.0,1,0,598.0,524.0,367.0,531.730320,477.321321
3430,3122,600.0,2,2,1.0,9.1834,8.0,1,0,221.0,369.0,590.0,522.031497,515.950036


In [23]:
y3 = df['price_2024']
x3 = df.iloc[:, [11, 12]]

x3_train, x3_test, y3_train, y3_test = train_test_split(x3, y3, test_size=0.28, random_state=42)
fit3 = MLPRegressor(hidden_layer_sizes=(100,50), activation='relu',solver='lbfgs',alpha=0.01,max_iter=1000).fit(x3_train,y3_train)
y3_pred = fit3.predict(x3)
df['price_2025'] = y3_pred
df

/Users/Shron/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


,postcode,price,bed,bath,parking,cbd_dist,school_counts,['f']_0,['f']_1,pop23-22,pop24-23,pop25-24,price_2023,price_2024,price_2025
0,3141,400.0,1,1,1.0,5.9614,6.0,1,0,-7.0,4.0,8.0,434.459686,402.722694,419.361688
1,3182,360.0,1,1,1.0,4.1882,5.0,1,0,151.0,219.0,170.0,383.762866,374.474120,378.368298
2,3141,320.0,1,1,0.0,6.9114,6.0,1,0,-7.0,4.0,8.0,388.895639,349.129574,387.639085
3,3163,360.0,1,1,1.0,14.3864,5.0,1,0,266.0,282.0,280.0,327.921215,377.099980,330.584414
4,3182,320.0,1,1,1.0,5.8505,5.0,1,0,151.0,219.0,170.0,377.294283,358.552649,374.499612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3427,3754,350.0,2,1,1.0,37.3377,10.0,1,0,5.0,6.0,17.0,431.572485,369.260675,419.495130
3428,3186,510.0,2,1,1.0,8.7118,8.0,1,0,225.0,255.0,260.0,500.945105,445.206938,470.345463
3429,3181,550.0,2,1,0.0,7.0455,3.0,1,0,598.0,524.0,367.0,531.730320,477.321321,493.855626
3430,3122,600.0,2,2,1.0,9.1834,8.0,1,0,221.0,369.0,590.0,522.031497,515.950036,478.108258


In [24]:
rate_df = df.iloc[:, [0, 1, 2, 12, 13, 14]]

rate_df['rate_22-23'] = (rate_df['price_2023'] - rate_df['price']) / rate_df['price']
rate_df['rate_23-24'] = (rate_df['price_2024'] - rate_df['price_2023']) / rate_df['price_2023']
rate_df['rate_24-25'] = (rate_df['price_2025'] - rate_df['price_2024']) / rate_df['price_2024']
rate_df['avg_rate'] = (rate_df['rate_22-23'] + rate_df['rate_23-24'] + rate_df['rate_24-25']) / 3

rate_df

<ipython-input-24-9a67370dd8c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rate_df['rate_22-23'] = (rate_df['price_2023'] - rate_df['price']) / rate_df['price']
<ipython-input-24-9a67370dd8c8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rate_df['rate_23-24'] = (rate_df['price_2024'] - rate_df['price_2023']) / rate_df['price_2023']
<ipython-input-24-9a67370dd8c8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,postcode,price,bed,price_2023,price_2024,price_2025,rate_22-23,rate_23-24,rate_24-25,avg_rate
0,3141,400.0,1,434.459686,402.722694,419.361688,0.086149,-0.073049,0.041316,0.018139
1,3182,360.0,1,383.762866,374.474120,378.368298,0.066008,-0.024204,0.010399,0.017401
2,3141,320.0,1,388.895639,349.129574,387.639085,0.215299,-0.102254,0.110301,0.074449
3,3163,360.0,1,327.921215,377.099980,330.584414,-0.089108,0.149971,-0.123351,-0.020829
4,3182,320.0,1,377.294283,358.552649,374.499612,0.179045,-0.049674,0.044476,0.057949
...,...,...,...,...,...,...,...,...,...,...
3427,3754,350.0,2,431.572485,369.260675,419.495130,0.233064,-0.144383,0.136041,0.074907
3428,3186,510.0,2,500.945105,445.206938,470.345463,-0.017755,-0.111266,0.056465,-0.024185
3429,3181,550.0,2,531.730320,477.321321,493.855626,-0.033218,-0.102324,0.034640,-0.033634
3430,3122,600.0,2,522.031497,515.950036,478.108258,-0.129948,-0.011650,-0.073344,-0.071647


In [25]:
growth_df = rate_df.groupby('postcode')['avg_rate'].mean().sort_values(ascending = False)
growth_df.head(10)

postcode
3048    0.446252
3437    0.211757
3461    0.193791
3429    0.181859
3465    0.154987
3125    0.153603
3128    0.125156
3085    0.117408
3083    0.114320
3630    0.107603
Name: avg_rate, dtype: float64